In [1]:
"""""
#   SPLIT merged_data.csv → smaller CSV files (memory-safe)      


import pandas as pd, pathlib, time
from tqdm.notebook import tqdm

# paths & parameters 
CSV_PATH   = pathlib.Path("./merged_data.csv")    # 25 GB source
OUT_DIR    = pathlib.Path("csv_chunks")           # output folder
OUT_DIR.mkdir(exist_ok=True)

SAMPLE_ROWS = 100_000     # to estimate average row size
TARGET_MB   = 300         # aim ≈ this much RAM per chunk
BYTES_MB    = 1_048_576

# (optional) limit columns or set dtypes here to shrink memory
USECOLS  = None           # e.g. ["track_id","title","artist", ...]
DTYPES   = None           # e.g. {"streams":"float32", ...}

# estimate rows per chunk 
t0 = time.time()
sample = pd.read_csv(
    CSV_PATH,
    nrows=SAMPLE_ROWS,
    usecols=USECOLS,
    dtype=DTYPES,
    low_memory=True,
    encoding_errors="ignore"
)
row_bytes = sample.memory_usage(deep=True).sum() / SAMPLE_ROWS
ROWS_PER_CHUNK = int((TARGET_MB * BYTES_MB) / row_bytes)
del sample

print(f"► Target RAM~{TARGET_MB} MB → {ROWS_PER_CHUNK:,} rows/chunk "
      f"(est. {row_bytes/1024:.1f} KB per row, calc {time.time()-t0:.1f}s)")

# stream-read and write chunks
reader = pd.read_csv(
    CSV_PATH,
    chunksize=ROWS_PER_CHUNK,
    usecols=USECOLS,
    dtype=DTYPES,
    low_memory=True,
    encoding_errors="ignore"
)

for i, chunk in enumerate(tqdm(reader, desc="Splitting CSV"), 1):
    out_file = OUT_DIR / f"part_{i:03}.csv"
    chunk.to_csv(out_file, index=False)
    print(f"✓ {out_file.name:>10} · {len(chunk):,} rows")
    del chunk                     # free RAM

print("All parts saved in", OUT_DIR)
"""""


'""\n#   SPLIT merged_data.csv → smaller CSV files (memory-safe)      \n\n\nimport pandas as pd, pathlib, time\nfrom tqdm.notebook import tqdm\n\n# paths & parameters \nCSV_PATH   = pathlib.Path("./merged_data.csv")    # 25 GB source\nOUT_DIR    = pathlib.Path("csv_chunks")           # output folder\nOUT_DIR.mkdir(exist_ok=True)\n\nSAMPLE_ROWS = 100_000     # to estimate average row size\nTARGET_MB   = 300         # aim ≈ this much RAM per chunk\nBYTES_MB    = 1_048_576\n\n# (optional) limit columns or set dtypes here to shrink memory\nUSECOLS  = None           # e.g. ["track_id","title","artist", ...]\nDTYPES   = None           # e.g. {"streams":"float32", ...}\n\n# estimate rows per chunk \nt0 = time.time()\nsample = pd.read_csv(\n    CSV_PATH,\n    nrows=SAMPLE_ROWS,\n    usecols=USECOLS,\n    dtype=DTYPES,\n    low_memory=True,\n    encoding_errors="ignore"\n)\nrow_bytes = sample.memory_usage(deep=True).sum() / SAMPLE_ROWS\nROWS_PER_CHUNK = int((TARGET_MB * BYTES_MB) / row_bytes)\n

In [ ]:
SPOTIFY_CLIENT_ID = "i erased my id"
SPOTIFY_CLIENT_SECRET = "i erased my client secret"
GENIUS_ACCESS_TOKEN = "i erased my access token"

In [ ]:
# imports and folders 
import pandas as pd, pathlib, json, time, requests, re
from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials
import lyricsgenius
from tenacity import retry, stop_after_attempt, wait_exponential
from tqdm.notebook import tqdm

IN_DIR    = pathlib.Path("csv_chunks")
OUT_DIR   = pathlib.Path("csv_enriched");  OUT_DIR.mkdir(exist_ok=True)
CACHE_DIR = pathlib.Path(".cache_enrich"); CACHE_DIR.mkdir(exist_ok=True)

TARGET_KEEP = 15000           # final row count

#API sessions and caches 
sp = Spotify(auth_manager=SpotifyClientCredentials(
    client_id=SPOTIFY_CLIENT_ID, client_secret=SPOTIFY_CLIENT_SECRET))

genius = lyricsgenius.Genius(
    GENIUS_ACCESS_TOKEN, timeout=15, retries=3,
    skip_non_songs=True, remove_section_headers=True, verbose=False)

def _load(n):  p=CACHE_DIR/f"{n}.json"; return json.loads(p.read_text()) if p.exists() else {}
def _save(n,o): (CACHE_DIR/f"{n}.json").write_text(json.dumps(o))

lyrics_cache    = _load("lyrics")        # {track_id: lyrics or ""}
artist_cache    = _load("artist_ids")    # {track_id: artist_id}
followers_cache = _load("followers")     # {artist_id: followers}

#helpers 
@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=2))
def g_search(title, artist):
    return genius.search_song(title=title, artist=artist)

def clean_title(title: str) -> str:
    """Drop everything in parentheses and trim."""
    return re.sub(r"\s*\([^)]*\)", "", title).strip()

def main_artist(artist: str) -> str:
    """Take only the first (primary) artist before a comma."""
    return artist.split(",")[0].strip()

def get_lyrics(tid: str, title: str, artist: str) -> str:
    if tid in lyrics_cache:
        return lyrics_cache[tid]

    raw_title = clean_title(title)
    primary   = main_artist(artist)
    print(f"♪ Searching: {raw_title} – {primary}", end="")

    try:
        # bias toward original page with the word lyrics
        song = g_search(f"{raw_title}", primary+" english")

        if (
            not song or
            song.primary_artist.name.lower() != primary.lower() or
            any(k in song.title.lower() for k in ("translation", "çeviri", "traducción"))
        ):
            song = None

        lyrics_cache[tid] = song.lyrics[:4000] if song and song.lyrics else ""
        print("  →  FOUND with english" if lyrics_cache[tid] else "  →  NOT FOUND with english")
        if not lyrics_cache[tid]:
            song = g_search(f"{raw_title}", primary+" lyrics")

            if (
            not song or
            song.primary_artist.name.lower() != primary.lower() or
            any(k in song.title.lower() for k in ("translation", "çeviri", "traducción"))
            ):
                song = None

            lyrics_cache[tid] = song.lyrics[:4000] if song and song.lyrics else ""
            print("  →  FOUND with lyrics" if lyrics_cache[tid] else "  →  NOT FOUND with lyrics")
            if not lyrics_cache[tid]:
                
                song = g_search(f"{primary}", raw_title+" english")
                if (
                not song or
                song.primary_artist.name.lower() != primary.lower() or
                any(k in song.title.lower() for k in ("translation", "çeviri", "traducción"))
                ):
                    song = None

                lyrics_cache[tid] = song.lyrics[:4000] if song and song.lyrics else ""
                print("  →  FOUND with english backwards" if lyrics_cache[tid] else "  →  NOT FOUND with english backwards")
                if not lyrics_cache[tid]:
                    song = g_search(f"{primary}", raw_title+" lyrics")
                    if (
                    not song or
                    song.primary_artist.name.lower() != primary.lower() or
                    any(k in song.title.lower() for k in ("çeviri", "traducción"))
                    ):# Türkiye üzerinden çağırı yapıldığı için otomatik çeviriler harici söz alınması için
                        song = None

                    lyrics_cache[tid] = song.lyrics[:4000] if song and song.lyrics else ""
                    print("  →  FOUND with lyrics backwards" if lyrics_cache[tid] else "  →  NOT FOUND with lyrics backwards")
                if not lyrics_cache[tid]:
                    song = g_search(f"{primary}", raw_title)
                    if (
                    not song or
                    song.primary_artist.name.lower() != primary.lower() or
                    any(k in song.title.lower() for k in ("çeviri", "traducción"))
                    ):
                        song = None

                    lyrics_cache[tid] = song.lyrics[:4000] if song and song.lyrics else ""
                    print("  →  FOUND with raw" if lyrics_cache[tid] else "  →  NOT FOUND with raw")                                    
                
            
    except Exception as e:
        lyrics_cache[tid] = ""
        print(f"  →  NOT FOUND ({e})")
            
    _save("lyrics", lyrics_cache)
    time.sleep(0.7)            # Genius rate-limit
    return lyrics_cache[tid]

def get_followers(tid: str) -> int:
    if tid not in artist_cache:
        try:
            artist_cache[tid] = sp.track(tid)["artists"][0]["id"]
            _save("artist_ids", artist_cache)
        except Exception:
            artist_cache[tid] = ""
    aid = artist_cache[tid]

    if aid and aid not in followers_cache:
        try:
            followers_cache[aid] = sp.artist(aid)["followers"]["total"]
            _save("followers", followers_cache)
            time.sleep(0.2)
        except Exception:
            followers_cache[aid] = 0
    return followers_cache.get(aid, 0)


kept, seen = [], set()                     # seen: track_ids already kept

for src in tqdm(sorted(IN_DIR.glob("part_*.csv")), desc="files"):
    if len(kept) >= TARGET_KEEP:
        break

    df = pd.read_csv(
        src,
        dtype={"artist":"string","title":"string","track_id":"string","region":"string"},
        low_memory=False
    )

    valid_regions = ["United States", "Canada","United Kingdom","Australia"]          # istediğim ülkeler
    df = df[df["region"].isin(valid_regions) & (df["popularity"] > 0)]
    if df.empty:
        continue

    print(f"\n--- {src.name} ({len(df):,} US rows) ---")

    for row in df.itertuples(index=False):
        if len(kept) >= TARGET_KEEP:
            break

        tid = row.track_id
        if tid in seen:                    # duplicate track → skip
            continue

        title, artist = row.title, row.artist
        if pd.isna(title) or pd.isna(artist):
            continue

        lyr = get_lyrics(tid, title, artist)
        if not lyr:
            continue

        foll = get_followers(tid)
        if foll == 0:
            continue

        kept.append({**row._asdict(), "lyrics": lyr, "artist_followers": foll})
        seen.add(tid)

print(f"\n✅ collected {len(kept):,} unique tracks with lyrics & followers")

pd.DataFrame(kept).to_csv(OUT_DIR / "sample_US_10000.csv", index=False)
print("saved csv_enriched/sample_US_10000.csv")


files:   0%|          | 0/119 [00:00<?, ?it/s]


--- part_001.csv (12,279 US rows) ---

--- part_002.csv (10,905 US rows) ---

--- part_003.csv (13,402 US rows) ---

--- part_004.csv (13,445 US rows) ---

--- part_005.csv (13,793 US rows) ---

--- part_006.csv (12,618 US rows) ---

--- part_007.csv (14,262 US rows) ---

--- part_008.csv (12,433 US rows) ---

--- part_009.csv (13,274 US rows) ---

--- part_010.csv (13,352 US rows) ---

--- part_011.csv (14,183 US rows) ---

--- part_012.csv (13,047 US rows) ---

--- part_013.csv (13,590 US rows) ---

--- part_014.csv (13,924 US rows) ---

--- part_015.csv (14,498 US rows) ---

--- part_016.csv (14,877 US rows) ---

--- part_017.csv (14,641 US rows) ---
♪ Searching: I Miss You – Clean Bandit

c:\Users\4444e\anaconda3\Lib\site-packages\lyricsgenius\genius.py:468: FutureWarning: The constructor signature will change in a future version. It will change to Song(lyrics, body) instead of Song(client, json_dict, lyrics).
  song = Song(self, song_info, lyrics)


  →  FOUND with english
♪ Searching: La ziguezon – La Bottine Souriante  →  NOT FOUND with english
  →  NOT FOUND with lyrics
  →  NOT FOUND with english backwards
  →  NOT FOUND with lyrics backwards
  →  FOUND with raw
♪ Searching: TAlk tO Me - Remix – Tory Lanez  →  NOT FOUND with english
  →  NOT FOUND with lyrics
  →  NOT FOUND with english backwards
  →  NOT FOUND with lyrics backwards
  →  FOUND with raw
♪ Searching: can't leave without it – 21 Savage  →  FOUND with english
♪ Searching: MoshPit – Kodak Black  →  FOUND with english
♪ Searching: Come and Get Your Love - Single Version – Redbone  →  NOT FOUND with english
  →  NOT FOUND with lyrics
  →  NOT FOUND with english backwards
  →  NOT FOUND with lyrics backwards
  →  NOT FOUND with raw
♪ Searching: Calling My Spirit – Kodak Black  →  FOUND with english
♪ Searching: Hide – Juice WRLD  →  FOUND with english

--- part_018.csv (13,115 US rows) ---
♪ Searching: Photographs – Professor Green

c:\Users\4444e\anaconda3\Lib\site-packages\lyricsgenius\genius.py:468: FutureWarning: The constructor signature will change in a future version. It will change to Song(lyrics, body) instead of Song(client, json_dict, lyrics).
  song = Song(self, song_info, lyrics)


...

c:\Users\4444e\anaconda3\Lib\site-packages\lyricsgenius\genius.py:468: FutureWarning: The constructor signature will change in a future version. It will change to Song(lyrics, body) instead of Song(client, json_dict, lyrics).
  song = Song(self, song_info, lyrics)


  →  NOT FOUND with raw
♪ Searching: Hook, Line & Sinker – Royal Blood  →  NOT FOUND with english
  →  NOT FOUND with lyrics
  →  NOT FOUND with english backwards
  →  NOT FOUND with lyrics backwards
  →  FOUND with raw
♪ Searching: May I Have This Dance [feat. Chance the Rapper] – Francis and the Lights  →  NOT FOUND with english
  →  NOT FOUND with lyrics
  →  NOT FOUND with english backwards
  →  NOT FOUND with lyrics backwards
  →  NOT FOUND with raw
♪ Searching: Work from Home - Recorded at Spotify Studios NYC – Clean Bandit  →  NOT FOUND with english
  →  NOT FOUND with lyrics
  →  NOT FOUND with english backwards
  →  NOT FOUND with lyrics backwards
  →  NOT FOUND with raw
♪ Searching: X Men – Lil Yachty  →  FOUND with english
♪ Searching: Hooked – NOTION  →  NOT FOUND with english
  →  NOT FOUND with lyrics
  →  NOT FOUND with english backwards
  →  NOT FOUND with lyrics backwards
  →  FOUND with raw
♪ Searching: Ghost – Jauz  →  NOT FOUND with english
  →  NOT FOUND with lyric

c:\Users\4444e\anaconda3\Lib\site-packages\lyricsgenius\genius.py:468: FutureWarning: The constructor signature will change in a future version. It will change to Song(lyrics, body) instead of Song(client, json_dict, lyrics).
  song = Song(self, song_info, lyrics)


  →  FOUND with english
♪ Searching: It's My Life – Bon Jovi  →  FOUND with english
♪ Searching: Buck Rogers – Feeder  →  FOUND with english
♪ Searching: Never Do a Tango with an Eskimo – Alma Cogan  →  NOT FOUND with english
  →  NOT FOUND with lyrics
  →  NOT FOUND with english backwards
  →  NOT FOUND with lyrics backwards
  →  FOUND with raw
♪ Searching: O Holy Night – Adolphe Adam  →  FOUND with english
♪ Searching: shira – Glassjaw  →  NOT FOUND with english
  →  NOT FOUND with lyrics
  →  NOT FOUND with english backwards
  →  NOT FOUND with lyrics backwards
  →  FOUND with raw
♪ Searching: Millionaire – Chris Stapleton  →  FOUND with english
♪ Searching: Be Ok Again – What So Not  →  NOT FOUND with english
  →  NOT FOUND with lyrics
  →  NOT FOUND with english backwards
  →  NOT FOUND with lyrics backwards
  →  FOUND with raw
♪ Searching: Juice – CHISEKO  →  NOT FOUND with english
  →  NOT FOUND with lyrics
  →  NOT FOUND with english backwards
  →  NOT FOUND with lyrics backwar

KeyboardInterrupt: 

In [4]:
import pandas as pd
pd.DataFrame(kept)["track_id"].size

7612

In [5]:
# ──────────────────────────────
# 0. KÜTÜPHANELER
# ──────────────────────────────
from transformers import pipeline
import torch, pandas as pd, requests, time, re, string
from tqdm.auto import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# ──────────────────────────────
# 1. GoEmotions MODEL
# ──────────────────────────────
device = 0 if torch.cuda.is_available() else -1
emo_pipe = pipeline(
    "text-classification",
    model="SamLowe/roberta-base-go_emotions",
    top_k=None,
    truncation=True,
    device=device,
    framework="pt"
)

positive_tags = {"joy", "love", "excitement", "admiration"}
negative_tags = {"sadness", "anger", "fear"}
relevant_tags = sorted(list(positive_tags | negative_tags | {"neutral"}))


# ──────────────────────────────
# 3. LDA (scikit-learn) YARDIMCI
# ──────────────────────────────
def sklearn_topics(series, n_topics):
    """Verilen string Series için LDA -> en olası topic id dizisi döndürür."""
    vect = CountVectorizer(
        stop_words="english",
        token_pattern=r"\b[a-zA-Z]{4,}\b"   # ≥4 harfli kelimeler
    )
    X = vect.fit_transform(series.fillna(""))
    lda = LatentDirichletAllocation(
        n_components=n_topics,
        max_iter=10,
        learning_method="online",
        random_state=42
    ).fit(X)
    topic_ids = lda.transform(X).argmax(axis=1)  # her belge için argmax
    return topic_ids

# ──────────────────────────────
# 4. ANA PIPELINE
# ──────────────────────────────
def add_all_features(df, max_len=2000):
    df = df.copy()
    df["title_topic"]  = sklearn_topics(df["title"], n_topics=10)
    df["lyrics_topic"] = sklearn_topics(df["lyrics"], n_topics=20)

    emo_scores = {f"emo_{t}":[] for t in relevant_tags}
    g_label, g_score = [], []
    pos_list, neg_list, emo_int = [], [], []


    for _, row in tqdm(df.iterrows(), total=len(df), desc="GoEmotions + Deezer + LDA"):
        # ---- GoEmotions ----
        snippet = (row.get("lyrics") or "")[:max_len]
        pred = emo_pipe(snippet)[0] if snippet else [{"label":"neutral","score":1.0}]
        scores = {d["label"]:d["score"] for d in pred}

        pos = sum(scores.get(t,0) for t in positive_tags)
        neg = sum(scores.get(t,0) for t in negative_tags)
        neu = scores.get("neutral",0)
        emo_sum = pos + neg

        if max(pos,neg,neu) == pos:
            g_label.append("POSITIVE"); g_score.append(pos)
        elif max(pos,neg,neu) == neg:
            g_label.append("NEGATIVE"); g_score.append(neg)
        else:
            g_label.append("NEUTRAL");  g_score.append(neu)

        pos_list.append(pos)
        neg_list.append(neg)
        emo_int.append(emo_sum)

        for t in relevant_tags:
            emo_scores[f"emo_{t}"].append(scores.get(t,0))




        time.sleep(0.2)

    df["bert_sent_label"]      = g_label
    df["bert_sent_score"]      = g_score
    df["positivity"]           = pos_list
    df["negativity"]           = neg_list
    df["emotional_intensity"]  = emo_int


    for t in relevant_tags:
        df[f"bert_{t}"] = emo_scores[f"emo_{t}"]

    return df


Device set to use cuda:0


In [6]:

df_features = add_all_features(pd.DataFrame(kept))
df_features = df_features[df_features['lyrics'].notnull() & (df_features['lyrics'].str.strip() != '')]
df_features.to_excel("1507.xlsx", index=False)

GoEmotions + Deezer + LDA:   0%|          | 0/7612 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [8]:
df_features["track_id"].unique

<bound method Series.unique of 0       5aAx2yezTd8zXrkmtKl66Z
1       5knuzwU65gJK7IF5yJsuaW
2       7BKLCZ1jbUBVqRi2FVlTVw
3       78rIJddV4X0HkNAInEcYde
4       5uCax9HTNlzGybIStD3vDh
                 ...          
7607    5oyMCoYK2rRMjAvZUHbn7R
7608    1QO9k34FcHXGqdMUWjH1Rn
7609    0Sc5lXL9iqUHibXRfiquGn
7610    0y7Xf4d6snUqmMFrCTD3ql
7611    3YKAaNsQ9i5oiSPeDw5PxV
Name: track_id, Length: 7612, dtype: object>